# Demo: Auditoría de Calidad de Datos

Este script demuestra el funcionamiento del sistema de auditoría de calidad de datos.
El sistema permite:
- Análisis de valores nulos: Identificar y cuantificar datos faltantes
- Análisis de unicidad: Detectar duplicados y valores únicos
- Análisis estadístico: Calcular métricas descriptivas
- Análisis de fechas: Validar coherencia temporal
- Generación de informes: Crear reportes en múltiples formatos
- Configuración flexible: Reglas definidas en archivos YAML

>El sistema sigue principios de Clean Architecture y está diseñado para ser
extensible y mantenible.

## SECCIÓN 1: IMPORTS

In [ ]:
import sys
import json
from pathlib import Path

# ■■■■■■■■■■■■■ Añadir el directorio raíz al path para importar el módulo src ■■■■■■■■■■■■■
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# ■■■■■■■■■■■■■ Importar componentes del sistema ■■■■■■■■■■■■■
try:
    from src import (
        QualityAuditor,
        CSVReader,
        QualityRulesReader,
        QualityReport,
        CSVValidator,
        SchemaValidator
    )

    print("✅ Todos los componentes importados correctamente")
except ImportError as e:
    print(f"❌ Error al importar componentes: {e}")
    print(f"📁 Directorio actual: {Path().absolute()}")
    print(f"📁 Directorio raíz intentado: {project_root}")

    # ▲▲▲▲▲▲ Mostrar primeros paths ▲▲▲▲▲▲
    print(f"📁 Path actual: {sys.path[:3]}")

## SECCIÓN 2: CARGA DE DATOS

In [ ]:
print("\n" + "▢▣" * 20)
print("📊 CARGA DE DATOS")
print("▢▣" * 20)

# ■■■■■■■■■■■■■ Definir rutas de archivos ■■■■■■■■■■■■■
data_path = project_root / "data" / "input" / "sample_data.csv"
config_path = project_root / "schemas" / "quality_rules.yaml"
schema_path = project_root / "schemas" / "default_schema.yaml"

print(f"📁 Ruta de datos: {data_path}")
print(f"⚙️ Ruta de configuración: {config_path}")
print(f"📋 Ruta de esquema: {schema_path}")

# ■■■■■■■■■■■■■ Verificar que los archivos existen ■■■■■■■■■■■■■
if not data_path.exists():
    print(f"❌ Error: No se encuentra el archivo de datos en {data_path}")

if not config_path.exists():
    print(f"❌ Error: No se encuentra el archivo de configuración en {config_path}")

print("✅ Archivo de datos encontrado")
print("✅ Archivo de configuración encontrado")

# ■■■■■■■■■■■■■ Cargar datos usando CSVReader ■■■■■■■■■■■■■
try:
    reader = CSVReader()
    data = list(reader.read_rows(str(data_path)))

    print(f"\n📊 Datos cargados exitosamente:")
    print(f"   - Total de filas: {len(data)}")
    if data:
        print(f"   - Columnas: {list(data[0].keys())}")
        print(f"   - Primeras 3 filas:")
        for i, row in enumerate(data[:3]):
            print(f"     {i + 1}: {row}")

except Exception as e:
    print(f"❌ Error al cargar datos: {e}")

## SECCIÓN 3: AUDITORÍA BÁSICA

In [ ]:
print("\n" + "▢▣" * 20)
print("🔍 AUDITORÍA BÁSICA")
print("▢▣" * 20)

if not data:
    print("❌ No hay datos para analizar")

try:
    # ■■■■■■■■■■■■■ Ejecutar auditoría básica (sin configuración) ■■■■■■■■■■■■■
    print("🔍 Ejecutando auditoría básica...")
    basic_results = QualityAuditor.quality_audit(data)

    print("✅ Auditoría básica completada")
    print(f"📈 Resultados básicos:")
    print(f"   - Total de filas procesadas: {basic_results.get('total_rows', 'N/A')}")
    print(f"   - Análisis de nulos: {'✅' if 'null_analysis' in basic_results else '❌'}")
    print(f"   - Análisis de unicidad: {'✅' if 'uniqueness_analysis' in basic_results else '❌'}")
    print(f"   - Análisis estadístico: {'✅' if 'statistical_analysis' in basic_results else '❌'}")
    print(f"   - Análisis de fechas: {'✅' if 'date_analysis' in basic_results else '❌'}")

except Exception as e:
    print(f"❌ Error en auditoría básica: {e}")

## SECCIÓN 4: AUDITORÍA CON CONFIGURACIÓN

In [ ]:
print("\n" + "▢▣" * 20)
print("⚙️ AUDITORÍA CON CONFIGURACIÓN")
print("▢▣" * 20)

if not data or not config_path:
    print("❌ No hay datos o configuración no disponible")

try:
    # ■■■■■■■■■■■■■ Cargar configuración desde YAML ■■■■■■■■■■■■■
    print("⚙️ Cargando configuración desde YAML...")
    config = QualityRulesReader.load_configs(str(config_path))

    print("✅ Configuración cargada:")
    print(f"   - Secciones: {list(config.keys())}")
    if 'quality_rules' in config:
        rules = config['quality_rules']
        print(f"   - Reglas de calidad: {list(rules.keys())}")

    # ■■■■■■■■■■■■■ Ejecutar auditoría con configuración ■■■■■■■■■■■■■
    print("\n🔍 Ejecutando auditoría con configuración...")
    config_results = QualityAuditor.quality_audit(data, str(config_path))

    print("✅ Auditoría con configuración completada")
    print(f"📈 Resultados configurados:")
    print(f"   - Total de filas: {config_results.get('total_rows', 'N/A')}")
    print(f"   - Configuración aplicada: {config_results.get('config_applied', {})}")

except Exception as e:
    print(f"❌ Error en auditoría con configuración: {e}")

## Sección 5: Impresión de resultados

### 5.1. VISUALIZACIÓN DE RESULTADOS

In [ ]:
def display_results(results, title="Resultados de Auditoría"):
    """
    Muestra los resultados de la auditoría de forma clara y legible.
    :param results: Resultados de la auditoría
    :param title: Titulo de la visualizacion de resultados
    :return:
    """
    print("\n" + "▢▣" * 20)
    print(f"📊 {title}")
    print("▢▣" * 20)

    # ■■■■■■■■■■■■■ Información general ■■■■■■■■■■■■■
    total_rows = results.get('total_rows', 0)
    print(f"📋 Información General:")
    print(f"   - Total de filas: {total_rows}")

    # ■■■■■■■■■■■■■ Análisis de nulos ■■■■■■■■■■■■■
    if 'null_analysis' in results:
        null_analysis = results['null_analysis']
        print(f"\n🔍 Análisis de Nulos:")
        for column, info in null_analysis.items():
            if isinstance(info, dict):
                count = info.get('count', 0)
                percentage = info.get('percentage', 0)
                print(f"   - {column}: {count} nulos ({percentage:.1f}%)")

    # ■■■■■■■■■■■■■ Análisis de unicidad ■■■■■■■■■■■■■
    if 'uniqueness_analysis' in results:
        uniqueness = results['uniqueness_analysis']
        print(f"\n🎯 Análisis de Unicidad:")
        for column, info in uniqueness.items():
            if isinstance(info, dict):
                unique_count = info.get('unique_count', 0)
                percentage = info.get('unique_percentage', 0)
                classification = info.get('classification', 'unknown')
                print(f"   - {column}: {unique_count} únicos ({percentage:.1f}%) - {classification}")

    # ■■■■■■■■■■■■■ Análisis estadístico ■■■■■■■■■■■■■
    if 'statistical_analysis' in results:
        stats = results['statistical_analysis']
        print(f"\n📈 Análisis Estadístico:")
        for column, info in stats.items():
            if isinstance(info, dict) and 'numeric_stats' in info:
                num_stats = info['numeric_stats']
                print(f"   - {column}:")
                print(f"     * Media: {num_stats.get('mean', 'N/A'):.2f}")
                print(f"     * Mediana: {num_stats.get('median', 'N/A'):.2f}")
                print(f"     * Desviación estándar: {num_stats.get('std_dev', 'N/A'):.2f}")
                print(f"     * Mínimo: {num_stats.get('min', 'N/A')}")
                print(f"     * Máximo: {num_stats.get('max', 'N/A')}")

    # ■■■■■■■■■■■■■ Análisis de fechas ■■■■■■■■■■■■■
    if 'date_analysis' in results:
        dates = results['date_analysis']
        print(f"\n📅 Análisis de Fechas:")
        for column, info in dates.items():
            if isinstance(info, dict):
                valid_dates = info.get('valid_dates', 0)
                invalid_dates = info.get('invalid_dates', 0)
                date_range = info.get('date_range', {})
                print(f"   - {column}: {valid_dates} válidas, {invalid_dates} inválidas")
                if date_range.get('start') and date_range.get('end'):
                    print(f"     * Rango: {date_range['start']} a {date_range['end']}")

In [ ]:
def visualize_results(basic_results, config_results):
    """
    Muestra los resultados de ambas auditorías.
    :param basic_results: Resultados basicos
    :param config_results: Resultados con configuracion de auditoria de calidad
    :return:
    """
    print("\n" + "▢▣" * 20)
    print("📈 VISUALIZACIÓN DE RESULTADOS")
    print("▢▣" * 20)

    if basic_results:
        display_results(basic_results, "Auditoría Básica")

    if config_results and config_results != basic_results:
        display_results(config_results, "Auditoría con Configuración")

### 5.2. GENERACIÓN DE INFORME

In [ ]:
def generate_reports(results):
    """
    Crea informes en diferentes formatos usando los generadores de reportes del sistema.
    """
    print("\n" + "▢▣" * 20)
    print("📝 GENERACIÓN DE INFORME")
    print("▢▣" * 20)

    if not results:
        print("❌ No hay resultados para generar informes")

    try:
        print("📝 Generando informes...")

        # ■■■■■■■■■■■■■ Informe resumido ■■■■■■■■■■■■■
        summary_report = QualityReport.generate_summary_report(results)
        print("\n📋 Informe Resumido:")
        print("-" * 40)
        print(summary_report)

        # ■■■■■■■■■■■■■ Informe detallado ■■■■■■■■■■■■■
        detailed_report = QualityReport.generate_detail_report(results)
        print("\n📄 Informe Detallado (primeros 500 caracteres):")
        print("-" * 40)
        print(detailed_report[:500] + "..." if len(detailed_report) > 500 else detailed_report)

        # ■■■■■■■■■■■■■ Informe JSON ■■■■■■■■■■■■■
        json_report = QualityReport.generate_json_report(results)
        print("\n📊 Informe JSON (estructura):")
        print("-" * 40)
        json_data = json.loads(json_report)
        print(f"Claves principales: {list(json_data.keys())}")

        # ■■■■■■■■■■■■■ Guardar informes en archivos ■■■■■■■■■■■■■
        output_dir = project_root / "data" / "output"
        output_dir.mkdir(exist_ok=True)

        # ■■■■■■■■■■■■■ Guardar informe con timestamp ■■■■■■■■■■■■■
        saved_file = QualityReport.save_report_with_timestamp(
            results,
            f"{str(output_dir)}/demo_quality_report",
            "txt"
        )
        print(f"\n💾 Informe guardado en: {saved_file}")

    except Exception as e:
        print(f"❌ Error al generar informes: {e}")

### 5.3 ALERTAS

In [ ]:
def generate_alerts(results):
    """
    Genera alertas basadas en los resultados de la auditoría.
    :param results: Resultados
    :return:
    """
    alerts = []

    # ■■■■■■■■■■■■■ Alertas de nulos ■■■■■■■■■■■■■
    if 'null_analysis' in results:
        for column, info in results['null_analysis'].items():
            if isinstance(info, dict):
                percentage = info.get('percentage', 0)
                if percentage > 20:  # Umbral del 20%
                    alerts.append(f"⚠️ ALTA: Columna '{column}' tiene {percentage:.1f}% de valores nulos")
                elif percentage > 10:  # Umbral del 10%
                    alerts.append(f"⚠️ MEDIA: Columna '{column}' tiene {percentage:.1f}% de valores nulos")

    # ■■■■■■■■■■■■■ Alertas de unicidad ■■■■■■■■■■■■■
    if 'uniqueness_analysis' in results:
        for column, info in results['uniqueness_analysis'].items():
            if isinstance(info, dict):
                classification = info.get('classification', '')
                if classification == 'identifier':
                    alerts.append(f"✅ INFO: Columna '{column}' funciona como identificador único")
                elif classification == 'high_uniqueness':
                    alerts.append(f"📊 INFO: Columna '{column}' tiene alta unicidad")
                elif classification == 'low_uniqueness':
                    alerts.append(f"⚠️ BAJA: Columna '{column}' tiene baja unicidad (posible campo categórico)")

    # ■■■■■■■■■■■■■ Alertas generales ■■■■■■■■■■■■■
    total_rows = results.get('total_rows', 0)
    if total_rows == 0:
        alerts.append("❌ CRÍTICA: No hay datos para analizar")
    elif total_rows < 10:
        alerts.append(f"⚠️ BAJA: Dataset muy pequeño ({total_rows} filas)")

    return alerts

In [ ]:
def show_alerts(results):
    """
    Muestra las alertas generadas basadas en los resultados.
    :param results: Resultados de la auditoria.
    :return:
    """
    print("\n" + "▢▣" * 20)
    print("🚨 ALERTAS")
    print("▢▣" * 20)

    if not results:
        print("❌ No hay resultados para generar alertas")
        return

    print("🚨 Generando alertas...")
    alerts = generate_alerts(results)

    if alerts:
        print(f"\n📊 Se generaron {len(alerts)} alertas:")
        for i, alert in enumerate(alerts, 1):
            print(f"   {i}. {alert}")
    else:
        print("\n✅ No se generaron alertas (todo parece estar en orden)")

    # ■■■■■■■■■■■■■ También podemos usar la funcion de alertas del sistema si está disponible ■■■■■■■■■■■■■
    try:
        if hasattr(QualityReport, 'generate_alerts'):
            system_alerts = QualityReport.generate_alerts(results)
            if system_alerts:
                print(f"\n🔔 Alertas del sistema ({len(system_alerts)}):")
                for alert in system_alerts:
                    print(f"   - {alert}")

    # ■■■■■■■■■■■■■ La funcion podría no estar implementado ■■■■■■■■■■■■■
    except:
        pass

### 5.4. VALIDACIÓN CSV (OPCIONAL)

In [ ]:
def validate_csv(data_path, schema_path):
    """
    Demostración del sistema de validación de esquemas CSV.
    """
    print("\n" + "▢▣" * 20)
    print("🔍 VALIDACIÓN CSV (OPCIONAL)")
    print("▢▣" * 20)

    if not schema_path.exists() or not data_path.exists():
        print("⏭️ Saltando validación CSV (archivos no disponibles)")
        return

    try:
        print("🔍 Ejecutando validación de esquema CSV...")

        # ■■■■■■■■■■■■■ Cargar esquema YAML ■■■■■■■■■■■■■
        from readers.quality_rules_reader import QualityRulesReader
        schema = QualityRulesReader.load_configs(str(schema_path))

        print(f"📋 Esquema cargado: {list(schema.keys())}")

        # ■■■■■■■■■■■■■ Validar estructura del esquema ■■■■■■■■■■■■■
        schema_validator = SchemaValidator()
        is_valid = schema_validator.validate_schema_structure(schema)

        if is_valid:
            print("✅ Estructura del esquema válida")

            # ■■■■■■■■■■■■■ Obtener información del esquema ■■■■■■■■■■■■■
            required_fields = schema_validator.get_required_fields(schema)
            all_fields = schema_validator.get_all_field_names(schema)

            print(f"   - Campos totales: {len(all_fields)}")
            print(f"   - Campos requeridos: {len(required_fields)}")
            print(f"   - Campos obligatorios: {required_fields}")

            # ■■■■■■■■■■■■■ Validar archivo CSV contra esquema ■■■■■■■■■■■■■
            csv_validator = CSVValidator()
            validation_errors = csv_validator.validate_file(str(data_path), schema)

            if validation_errors:
                print(f"\n❌ Se encontraron {len(validation_errors)} errores de validación:")
                for i, error in enumerate(validation_errors, 1):
                    print(f"   {i}. {error}")
            else:
                print("\n✅ Validación CSV exitosa - No se encontraron errores")
        else:
            print("❌ El esquema tiene errores de estructura")

    except Exception as e:
        print(f"❌ Error en validación CSV: {e}")

## Sección 6: Función principal

In [ ]:
print("🚀 INICIANDO DEMOSTRACIÓN DE AUDITORÍA DE CALIDAD DE DATOS")
print("▢▣" * 20)

# ■■■■■■■■■■■■■ Validación de esquema CSV (opcional) ■■■■■■■■■■■■■
validate_csv(project_root / "data" / "input" / "sample_data.csv", schema_path)

if not data:
    print("❌ No se pudieron cargar los datos. Abortando demostración.")

# ■■■■■■■■■■■■■ Usar los resultados configurados si están disponibles, si no los básicos ■■■■■■■■■■■■■
main_results = config_results if config_results else basic_results

# ■■■■■■■■■■■■■ Visualizar resultados ■■■■■■■■■■■■■
visualize_results(basic_results, config_results)

# ■■■■■■■■■■■■■ Generar informes ■■■■■■■■■■■■■
generate_reports(main_results)

# ■■■■■■■■■■■■■ Mostrar alertas ■■■■■■■■■■■■■
show_alerts(main_results)

print("\n🎉 DEMOSTRACIÓN COMPLETADA EXITOSAMENTE")

## Conclusiones

En esta demostración hemos mostrado:

1. ✅ Importación de Componentes: Carga exitosa de todas las clases principales del sistema
2. 📊 Carga de Datos: Lectura de archivos CSV usando el lector especializado
3. 🔍 Auditoría Básica: Análisis de calidad sin configuración personalizada
4. ⚙️ Auditoría con Configuración: Análisis avanzado usando reglas YAML
5. 📈 Visualización de Resultados: Presentación clara de métricas y análisis
6. 📝 Generación de Informes: Creación de reportes en múltiples formatos
7. 🚨 Sistema de Alertas: Detección automática de problemas de calidad
8. ✅ Validación CSV: Verificación estructural contra esquemas (opcional)

### Características Clave del Sistema

- 🏗️ Arquitectura Modular: Cada componente tiene una responsabilidad clara
- ⚙️ Configuración Flexible: Reglas externas en archivos YAML
- 📊 Análisis Completo: Nulos, unicidad, estadísticas, fechas y más
- 📝 Informes Variados: TXT, JSON, HTML y formatos personalizados
- 🔍 Detección de Problemas: Alertas automáticas basadas en umbrales
- ✅ Testing FIRST: Sistema robusto con pruebas completas
- 🧹 Clean Code: Código legible y mantenible

### Próximos Pasos

- Extender Analizadores: Añadir nuevos tipos de análisis
- Personalizar Configuración: Crear reglas específicas para tus datos
- Integrar en Pipelines: Usar el sistema en flujos de ETL
- Automatizar Informes: Programar generación periódica de reportes

¡El sistema está listo para producción y puede adaptarse a tus necesidades específicas! 🚀